# Testing using widgets in HOPP

Attempting to use widgets to create a user-friendly way to test simulate_hybrid.py

### Load necessary modules

In [33]:
import json
import ipywidgets as widgets
from pathlib import Path

from hybrid.sites import SiteInfo, flatirons_site
from hybrid.hybrid_simulation import HybridSimulation
from hybrid.log import hybrid_logger as logger
from hybrid.keys import set_nrel_key_dot_env

## Set API Key

### This step gives you access to wind and solar resource data that will be fetched automatically for your desired location.

#### N.B. If you need an NREL API key, please get one here: https://developer.nrel.gov/signup/

In [34]:
set_nrel_key_dot_env()

### Step 1: Path

In [35]:
examples_dir = Path("__file__").parent.absolute()

### Set wind, solar, and interconnection capacities (in MW)

Solar capacity

In [56]:
solar = widgets.IntSlider(
    min=0,
    max=100,
    step=5,
    description='Solar [MW]:',
    value=50
)
wind = widgets.IntSlider(
    min=0,
    max=100,
    step=5,
    description='Wind [MW]:',
    value=50
)
interconnect = widgets.IntSlider(
    min=0,
    max=100,
    step=5,
    description='Interconnection [MW]:',
    value=50
)
display(solar)
display(wind)
display(interconnect)

IntSlider(value=50, description='Solar [MW]:', step=5)

IntSlider(value=50, description='Wind [MW]:', step=5)

IntSlider(value=50, description='Interconnection [MW]:', step=5)

In [61]:
solar_size_mw = solar.value
wind_size_mw = wind.value
interconnection_size_mw = interconnect.value

technologies = {'pv': {
                    'system_capacity_kw': solar_size_mw * 1000
                },
                'wind': {
                    'num_turbines': 10,
                    'turbine_rating_kw': 2000
                }}

### Get resources

In [62]:
lat = flatirons_site['lat']
lon = flatirons_site['lon']
prices_file = examples_dir.parent / "resource_files" / "grid" / "pricing-data-2015-IronMtn-002_factors.csv"
site = SiteInfo(flatirons_site, grid_resource_file=prices_file)

### Create Model

In [63]:
hybrid_plant = HybridSimulation(technologies, site, interconnect_kw=interconnection_size_mw * 1000)

hybrid_plant.pv.system_capacity_kw = solar_size_mw * 1000
hybrid_plant.wind.system_capacity_by_num_turbines(wind_size_mw * 1000)
hybrid_plant.ppa_price = 0.1
hybrid_plant.pv.dc_degradation = [0] * 25
hybrid_plant.simulate(25)

# Save the outputs
annual_energies = hybrid_plant.annual_energies
wind_plus_solar_npv = hybrid_plant.net_present_values.wind + hybrid_plant.net_present_values.pv
npvs = hybrid_plant.net_present_values

wind_installed_cost = hybrid_plant.wind.total_installed_cost
solar_installed_cost = hybrid_plant.pv.total_installed_cost
hybrid_installed_cost = hybrid_plant.grid.total_installed_cost

# Print Results
print("Wind Installed Cost: {}".format(wind_installed_cost))
print("Solar Installed Cost: {}".format(solar_installed_cost))
print("Hybrid Installed Cost: {}".format(hybrid_installed_cost))
print("Wind NPV: {}".format(hybrid_plant.net_present_values.wind))
print("Solar NPV: {}".format(hybrid_plant.net_present_values.pv))
print("Hybrid NPV: {}".format(hybrid_plant.net_present_values.hybrid))
print("Wind + Solar Expected NPV: {}".format(wind_plus_solar_npv))


print("Annual energy produced by: ", annual_energies)
print("Net present value of eache energy: ",npvs)

Dispatch optimization not required...
Wind Installed Cost: 72700000.0
Solar Installed Cost: 48000000.0
Hybrid Installed Cost: 120700000.0
Wind NPV: 57563093.65754582
Solar NPV: 42421773.18257575
Hybrid NPV: 96962596.21262509
Wind + Solar Expected NPV: 99984866.84012157
Annual energy produced by:  {"pv": 104053614.17869328, "wind": 154488744.61423218, "hybrid": 255073908.81025952}
Net present value of eache energy:  {"pv": 42421773.18257575, "wind": 57563093.65754582, "hybrid": 96962596.21262509}
